## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-06-41-00 +0000,wsj,"Gold Futures May Undergo Short-Term Rebound, C...",https://www.wsj.com/finance/commodities-future...
1,2025-11-19-06-25-18 +0000,bbc,'I volunteer at the foodbank that helped me',https://www.bbc.com/news/articles/c0exv9y9zeqo...
2,2025-11-19-06-22-00 +0000,wsj,Platinum Market Forecast to Recover With Small...,https://www.wsj.com/finance/commodities-future...
3,2025-11-19-06-20-56 +0000,nypost,Veterans exposed to toxic chemicals at Camp Le...,https://nypost.com/2025/11/19/us-news/veterans...
4,2025-11-19-06-16-38 +0000,nyt,Harvard Opens New Investigation Into Summers a...,https://www.nytimes.com/2025/11/19/us/harvard-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,59
43,epstein,27
40,new,20
62,saudi,17
88,files,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...,158
299,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...,144
145,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,140
11,2025-11-19-05-46-46 +0000,bbc,Congress approves bill to release Epstein file...,https://www.bbc.com/news/articles/cdxr1r34ev9o...,139
72,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...,133


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,158,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
299,85,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...
145,54,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
307,41,2025-11-18-12-39-21 +0000,nypost,House Democrat pushing for release of full Eps...,https://nypost.com/2025/11/18/us-news/rep-ro-k...
155,41,2025-11-18-21-46-47 +0000,latimes,"'Played with fire, got burned': GOP control of...",https://www.latimes.com/politics/story/2025-11...
205,40,2025-11-18-18-52-13 +0000,nypost,Google’s Sundar Pichai warns of AI spending ‘i...,https://nypost.com/2025/11/18/business/google-...
142,34,2025-11-18-22-19-58 +0000,bbc,Lebanon says Israeli strike killed 13 people n...,https://www.bbc.com/news/articles/c0l73j781y9o...
231,34,2025-11-18-17-18-09 +0000,nypost,NY Gov. Kathy Hochul enlists Wall Street execs...,https://nypost.com/2025/11/18/business/ny-gov-...
93,34,2025-11-19-00-42-46 +0000,nypost,Ex-wife of top NASA astronaut pleads guilty to...,https://nypost.com/2025/11/18/us-news/estrange...
259,33,2025-11-18-15-52-27 +0000,bbc,Two Ukrainians working for Russia behind rail ...,https://www.bbc.com/news/articles/c4gknv8nxlzo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
